In [1]:
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))
import pandas as  pd
# rawdata = pd.read_csv("test2.csv", sep=',', header = None, engine='python')

D:\Unity\Transporters
['.ipynb_checkpoints', 'Analyze_Specialization.ipynb', 'Analyze_transporter_output.asv', 'Analyze_transporter_output.m', 'CumulativeR.jpg', 'episode_rewards.csv', 'episode_rewards_agent0.csv', 'episode_rewards_agent1.csv', 'getting-started.ipynb', 'mode_freq_episode.csv', 'mode_freq_episode_agent0.csv', 'mode_freq_episode_agent1.csv', 'Reformat_Observation_data.ipynb', 'SD.jpg', 'SD0.jpg', 'SD1.jpg', 'TAgent_0.csv', 'TAgent_1.csv', 'TAgent_2.csv', 'TAgent_3.csv', 'test2.csv', 'Transporter_Academy.cs', 'Transporter_agent.cs', 'Transporter_Arena.cs']


In [65]:
import numpy as np
print(rawdata.shape[0]/3)
# print(rawdata.index[:])
# rawdata.drop(col, axis = 0, inplace = True)
# filter
# df = rawdata.loc[(rawdata.loc[range(100)][1] == 1)]
# df = rawdata[rawdata.loc[:][3]==1][3]
# df = rawdata.loc[3][np.arange(5)]
# df = rawdata.loc[3] #when you simply do rawdata[3] it returns the column
# df = rawdata.loc[:][3].isin([1,2,3])
# print(df)
# df = rawdata

16048.0


In [81]:
#create a new dataframe with the rows shifted so one row contains all observations instead of three
df = pd.DataFrame(index=np.arange(0,16049), columns = np.arange(0, (12+35*2)))
df.fillna(0)
df.shape
df_index=0
#takes a long time as is 
for index in np.arange(2, rawdata.shape[0], 3): #.iterrows():
    df.loc[df_index,:] = pd.concat([rawdata.loc[index,0:11], rawdata.loc[index-1,:], rawdata.loc[index-2,:]], axis = 0).to_numpy()
    df_index += 1
#     print(df.loc[index,:].shape)

print(df)

             0            1          2    3    4    5    6    7    8    9   \
0       0.62932            0  -0.777146    1    0    0    0    0    0    1   
1      0.469472            0   0.882948    1    0    0    0    0    1    0   
2      0.994522            0   0.104528    1    0    0    1    0    0    0   
3      0.615661            0  -0.788011    1    0    0    0    1    0    0   
4      0.241922            0  -0.970296    1    0    0    0    0    0    1   
...         ...          ...        ...  ...  ...  ...  ...  ...  ...  ...   
16044 -0.531183 -0.000305261  -0.847258    1    0    0    0    0    0    1   
16045 -0.618797  0.000525875  -0.785551    1    0    0    0    0    1    0   
16046 -0.998813   0.00186561  0.0486846    1    0    0    1    0    0    0   
16047   0.99879   -0.0425219 -0.0247087    0    0    1    0    1    0    0   
16048       NaN          NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

       ...   72   73   74   75        76   77   78   79   80   

In [2]:
# now we filter out only the rows for agent p 
# nullrows=df.loc[~df.iloc[:,1].isnull(), :].shape
# print(nullrows)
for agent_id in range(4):
    df_p = df.loc[df.loc[:,6+agent_id]==1,:]
    
    ##remove entire one-hot ID section since we are storing each agent as a separate file after this
    df_p.drop([6,7,8,9,10,11], axis="columns", inplace = True)
    df_p.to_csv('TAgent_' + str(agent_id) + '.csv', header= False, index = False)
    

(4012, 76)

In [93]:
#note i forgot i only ran the data with 4 agents not 6 so discard the columns 10,11
df_p = pd.read_csv("TAgent_2.csv", sep=',', header = None, engine='python')
df_p.shape

In [3]:
import pandas as pd
import numpy as np
# perform t-sne on data
df_p = pd.read_csv("TAgent_2.csv", sep=',', header = None, engine='python')
from sklearn.manifold import TSNE

#remember to decimate data before tsne!

# p_data = np.array(df_p.loc[::10,:])
p_data = np.array(df_p)

p_data_emb = TSNE(n_components=3, 
                  perplexity=30.0, 
                  early_exaggeration=24.0, 
                  learning_rate=200.0, 
                  n_iter=600, 
                  n_iter_without_progress = 51,
                  metric='euclidean', 
                  init='pca', 
                  verbose=1).fit(p_data)
p_data_emb.embedding_.shape

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 4012 samples in 0.031s...
[t-SNE] Computed neighbors for 4012 samples in 2.249s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4012
[t-SNE] Computed conditional probabilities for sample 2000 / 4012
[t-SNE] Computed conditional probabilities for sample 3000 / 4012
[t-SNE] Computed conditional probabilities for sample 4000 / 4012
[t-SNE] Computed conditional probabilities for sample 4012 / 4012
[t-SNE] Mean sigma: 0.673662
[t-SNE] KL divergence after 250 iterations with early exaggeration: 182.607681
[t-SNE] KL divergence after 1000 iterations: 0.887297


(4012, 3)

In [4]:
#try PCA 
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
# perform t-sne on data
df_p = pd.read_csv("TAgent_1.csv", sep=',', header = None, engine='python')

p_data = np.array(df_p)
pca = PCA(n_components=3, copy=True, whiten=False, svd_solver='auto', iterated_power='auto')
emb = pca.fit_transform(p_data)

In [1]:
#try isomap
import pandas as pd
import numpy as np
# perform t-sne on data
df_p = pd.read_csv("TAgent_0.csv", sep=',', header = None, engine='python')
from sklearn.manifold import Isomap
p_data = np.array(df_p)

isofit = Isomap(n_components=3, 
                  n_neighbors=6, 
                  eigen_solver='auto', 
                  path_method='D',
                  neighbors_algorithm='kd_tree', 
                  metric='euclidean').fit(p_data)
emb = isofit.embedding_

In [16]:
## get row indicies of the item carried etc and color by when item carried was 1 etc
item_carried = np.zeros((p_data.shape[0]))
item_carried[p_data[:,3]==1] = 0.0
item_carried[p_data[:,4]==1] = 1.0
item_carried[p_data[:,5]==1] = 2.0
# print(item_carried[200:300])

heading = np.zeros((p_data.shape[0], 1))
# heading = np.arctan(p_data[:,2]/p_data[:,0])
heading = p_data[:,0] #the x component of the forward facing vector
print(heading)
print(item_carried)



[ 0.9945219  0.9455187  0.7193397 ... -0.9988166 -0.8846806 -0.9988126]
[0. 0. 0. ... 0. 0. 0.]


In [26]:
# import matplotlib.pyplot as plt
# emb = p_data_emb.embedding_ #if using TSNE
import plotly
# Color scale for Scatter Plots with Graph Objects
import plotly.graph_objects as go

# fig = go.Figure()

# Create list from 0 to 39 to use as x, y, and color
# values = list(range(len(item_carried)))


# def toint(n):
#     return int(n)

# values=list(map(toint, item_carried))
values = item_carried
# values = heading

# print(values[1].dtype)
# print(values2[1].dtype)

fig = go.Figure(data=[go.Scatter3d(
    x=emb[:,0],
    y=emb[:,1],
    z=emb[:,2],
    mode='markers',
    marker=dict(
        size=2,
        color=values,                # set color to an array/list of desired values
#         colorscale=plotly.colors.cyclical.IceFire, 
        colorscale=plotly.colors.sequential.Jet,
#         opacity=0.7,
        colorbar=dict(title="Colorbar"),
        line=dict(color='DarkSlateGrey',width=.1)
    ) 
        
)])

fig.show()



In [5]:
fig.write_image("agent0_item.pdf")

In [192]:
# import plotly.express as px
# df_emb = pd.DataFrame(data = emb)
# df_heading = pd.DataFrame(data = heading)
# df_size = pd.DataFrame(data = 0.1*np.ones(len(heading)))

# df_emb = pd.concat([df_emb, df_heading, df_size], axis =1)
# df_emb.columns = [0,1,2,3, 4]
# print(df_emb.columns)

# fig = px.scatter_3d(df_emb, x=0, y=1, z=2, color=3, size=4, opacity=0.1)
# fig.show()

In [32]:
# D Scatter Plot with Colorscaling and Marker Styling

import plotly.graph_objects as go
import numpy as np
import plotly

# Helix equation
# t = np.linspace(0, 20, 100)
# x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(
    x=emb[:,0],
    y=emb[:,1],
    z=emb[:,2],
    mode='markers',
    marker=dict(
        size=2,
        color=heading,                # set color to an array/list of desired values
#         colorscale=plotly.colors.cyclical.IceFire, 
        colorscale=plotly.colors.sequential.Inferno,
#         opacity=0.6,
        colorbar=dict(title="X component of body orientation"),
        line=dict(color='DarkSlateGrey',width=.1)
    ) 
        
)])

# tight layout
# fig.update_layout(margin=dict(l=0, r=0, b=0))
fig.show()



In [200]:
p_data.shape
82-6

76